In [37]:
import pandas as pd
import numpy as np

## Собственная реализация решающего дерева для классификации

In [38]:
from sklearn.base import BaseEstimator, ClassifierMixin
from numba import njit
class Node:
    __slots__ = ('feature', 'threshold', 'left', 'right', 'value')
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature      # индекс признака для разбиения (у внутреннего узла)
        self.threshold = threshold  # порог разбиения
        self.left = left            # левое поддерево (Node)
        self.right = right          # правое поддерево (Node)
        self.value = value 
        
class MyDecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth = 5, min_samples_split = 2, min_samples_leaf = 1, criteria = 'gini'):
        self.root = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criteria = self.gini_criteria

    @staticmethod
    @njit
    def gini_criteria(total_by_class, total):
        if total == 0:
            return 0.0
        parts = total_by_class / total
        return 1 - np.sum(parts ** 2)


    '''
    1. Перебираем признаки
    2. Получаем массив значений каждого признака у объектов
    3. Сортируем его, сортируем таргет по этим индексам
    4. Сначала слева - 0 элементов, справа - все
    5. Бежим слева направо, после нахождения каждой группы одинаковых элементов считаем критерий качества данного разделения
    6. Запоминаем лучший
    '''

    # indices - индексы элементов, которые входят в текущий узел

    @staticmethod
    @njit
    def best_split_for_feature(sorted_features, sorted_labels, parent_score, n_classes, min_samples_leaf, criteria):
        n_samples = len(sorted_features)
        best_threshold = None
        best_split = parent_score
        left_border = 0

        right_counts = np.bincount(sorted_labels, minlength = n_classes)
        right_volume = n_samples
        
        left_counts = np.zeros_like(right_counts)
        left_volume = 0
        
        while left_border < n_samples - 1:
            start_value = sorted_features[left_border]
            j = left_border
            while j < n_samples and sorted_features[j] == start_value:
                right_counts[sorted_labels[j]] -= 1
                left_counts[sorted_labels[j]] += 1
                right_volume -= 1
                left_volume += 1
                j+=1
                
            if left_volume >= min_samples_leaf and right_volume >= min_samples_leaf:
                criteria_left = criteria(left_counts, left_volume)
                criteria_right = criteria(right_counts, right_volume)
                current_score = left_volume / n_samples * criteria_left + right_volume / n_samples * criteria_right

                
                if current_score < best_split:
                    best_split = current_score
                    if j < n_samples:
                        best_threshold = (sorted_features[j] + start_value) / 2.0
                    else:
                        best_threshold = start_value
                        
            left_border = j
        return best_threshold, best_split
                
    def find_best_split(self, X, y, indices, parent_score):
        best_feature, best_threshold = None, None
        best_split = parent_score
        
        n_samples = len(indices)
        labels = y[indices]
        
        if n_samples < self.min_samples_leaf:
            return best_feature, best_threshold
        
        for feature in range(X.shape[1]):
            feature_values = X[indices, feature]

            sorted_feature_idx = np.argsort(feature_values)
            sorted_features = feature_values[sorted_feature_idx]
            sorted_labels = labels[sorted_feature_idx]

            current_threshold, current_split = self.best_split_for_feature(sorted_features, sorted_labels, parent_score, self.n_classes, self.min_samples_leaf, self.criteria)
            if current_split < best_split:
                best_feature = feature
                best_threshold = current_threshold
                best_split = current_split
        return best_feature, best_threshold
        
    def build_tree(self, X, y, indices, depth):
        n_samples = len(indices)
        n_classes = len(np.unique(y[indices]))

        if (n_classes == 1) or (depth >= self.max_depth) or (n_samples<self.min_samples_split):
            most_common = np.bincount(y[indices]).argmax()
            return Node(value = most_common)

        parent_counts = np.bincount(y[indices], minlength = self.n_classes)
        parent_score = self.criteria(parent_counts, n_samples)
        
        best_feature, best_threshold = self.find_best_split(X, y, indices, parent_score)
        
        if best_feature is None:  # не удалось найти разбиение
            most_common = np.bincount(y[indices]).argmax()
            return Node(value=most_common)
            
        feature_values = X[indices, best_feature]
        left_indices = indices[feature_values <= best_threshold]
        right_indices = indices[feature_values > best_threshold]

        left_subtree = self.build_tree(X, y, left_indices, depth+1)
        right_subtree = self.build_tree(X, y, right_indices, depth+1)
        
        return Node(feature = best_feature, threshold = best_threshold, left = left_subtree, right = right_subtree)
        
    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.n_classes = len(np.unique(y))
        self.root = self.build_tree(X, y, indices = np.arange(X.shape[0]), depth = 0)
        return self
        
    def predict(self, X):
        X = np.array(X)    
        return np.array([self.predict_one(x, self.root) for x in X])

    def predict_one(self, obj, node):
        if node.value is not None:
            return node.value
        if obj[node.feature] <= node.threshold:
            return self.predict_one(obj, node.left)
        return self.predict_one(obj, node.right)        

## Замерим качество работы

In [39]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
y = data.target
print(data.feature_names)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


### Моя реализация

In [40]:
%%time
from sklearn.model_selection import cross_val_score
my_tree_classifier = MyDecisionTreeClassifier()
print(f'Accuracy on cross_validation, (number of folds is 7): {np.mean(cross_val_score(my_tree_classifier, X, y, cv = 7, scoring = 'accuracy'))}')

Accuracy on cross_validation, (number of folds is 7): 0.9140104099453693
CPU times: total: 1.27 s
Wall time: 1.32 s


### Готовая реализация в sklearn

In [41]:
%%time
from sklearn.tree import DecisionTreeClassifier
tree_classifier = DecisionTreeClassifier(max_depth = 5)
print(f'Accuracy on cross_validation, (number of folds is 7): {np.mean(cross_val_score(tree_classifier, X, y, cv = 7, scoring = 'accuracy'))}')

Accuracy on cross_validation, (number of folds is 7): 0.9350238740482641
CPU times: total: 125 ms
Wall time: 135 ms
